# **Laboratorio 12: 🚀 Despliegue 🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Herrera Suárez
- Nombre de alumno 2: Hecmar Taucare Reyes

### **Link de repositorio de GitHub:** [Repositorio](https://github.com/Joaquin-HS/MDS7202)

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow (2.0 puntos)**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `f1-score` se registre como `"valid_f1"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

In [ ]:
import os
import pickle
import optuna
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from optuna.visualization.matplotlib import plot_optimization_history, plot_param_importances
import matplotlib.pyplot as plt
import xgboost as xgb

def optimize_model():
    # Se crean los directorios para los artefactos
    os.makedirs("plots", exist_ok=True)
    os.makedirs("models", exist_ok=True)

    # Se cargan los datos
    data = pd.read_csv("water_potability.csv")
    X = data.drop("Potability", axis=1)
    y = data["Potability"]

    # Se dividen los datos en entrenamiento y validación
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

    def objective(trial):
        # Grilla de parámetros del modelo
        params = {
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 0, 5),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        }

        # Nombre único para el experimento y el run
        experiment_name = f"XGBoost Experiment: lr_{params['learning_rate']:.3f}"
        mlflow.set_experiment(experiment_name)

        with mlflow.start_run(run_name=f"Run max_depth={params['max_depth']}"):
            # Se entrena el modelo
            model = XGBClassifier(**params, use_label_encoder=False, eval_metric="logloss")
            model.fit(X_train, y_train)

            # Se valida el modelo
            y_pred = model.predict(X_valid)
            f1 = f1_score(y_valid, y_pred)

            # Se registran parámetros y métricas en MLflow
            mlflow.log_params(params)
            mlflow.log_metric("valid_f1", f1)

            # Se registra el modelo en MLflow
            mlflow.sklearn.log_model(model, "model")

            return f1

    # Se optimiza con Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)

    # Se guardan los mejores resultados y gráficos
    best_trial = study.best_trial
    best_experiment_name = f"XGBoost Experiment: lr_{best_trial.params['learning_rate']:.3f}"
    experiment = mlflow.get_experiment_by_name(best_experiment_name)
    best_model = get_best_model(experiment.experiment_id)
    
    # Se inicia una run en el mejor experimento
    with mlflow.start_run(run_name="plots", experiment_id=experiment.experiment_id):
        # Gráfico de la importancia de las características del mejor modelo
        importance_fig = plot_feature_importance(best_model)
        importance_path = os.path.join("plots", "feature_importance.png")
        importance_fig.savefig(importance_path)
        
        # Se guardan los gráficos de Optuna e importancia de características en la run del mejor modelo
        mlflow.log_artifact(save_optuna_plot(plot_optimization_history(study), "optimization_history.png"))
        mlflow.log_artifact(save_optuna_plot(plot_param_importances(study), "param_importances.png"))
        mlflow.log_artifact(importance_path)
    
    # Se serializa el mejor modelo
    with open("models/best_model.pkl", "wb") as f:
        pickle.dump(best_model, f)

    print(f"Mejor F1-Score: {best_trial.value}")
    print(f"Mejores hiperparámetros: {best_trial.params}")


def save_optuna_plot(plot, filename):
    if isinstance(plot, plt.Axes):
        plot = plot.figure
    
    if isinstance(plot, plt.Figure):
        path = os.path.join("plots", filename)
        plot.savefig(path)
        return path
    else:
        raise ValueError("El argumento 'plot' debe ser una figura de Matplotlib.")


def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_ids=[experiment_id])
    best_model_id = runs.sort_values("metrics.valid_f1", ascending=False)["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model(f"runs:/{best_model_id}/model")
    return best_model


def plot_feature_importance(model):
    fig, ax = plt.subplots(figsize=(10, 8))
    xgb.plot_importance(model, importance_type="weight", max_num_features=10, ax=ax)
    plt.tight_layout()
    return fig


if __name__ == "__main__":
    optimize_model()

A continuación se muestran los resultados obtenidos de la optimización: <br>
![Resultados de la optimización](./imgs/results_optimize.png)

Y una visión general de la interfaz visual de mlflow: <br>
![mlflow1](./imgs/mlflow1.png) <br> <br>
![mlflow2](./imgs/mlflow2.png) <br> <br>
![mlflow2](./imgs/mlflow3.png)

# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

**`HINT:` Recuerde que puede utilizar [http://localhost:8000/docs](http://localhost:8000/docs) para hacer un `POST`.**

In [ ]:
# !pip install fastapi uvicorn

from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import numpy as np

# Se inicializa FastAPI
app = FastAPI()

# Estructura de entrada
class WaterSample(BaseModel):
    ph: float
    Hardness: float
    Solids: float
    Chloramines: float
    Sulfate: float
    Conductivity: float
    Organic_carbon: float
    Trihalomethanes: float
    Turbidity: float

# Se carga el modelo optimizado
def load_model():
    with open("models/best_model.pkl", "rb") as f:
        model = pickle.load(f)
    return model

model = load_model()

# Ruta GET para describir el modelo
@app.get("/")
def read_root():
    return {
        "description": "Modelo de clasificación de potabilidad del agua",
        "problem": "Predecir si el agua es potable basándose en varias características fisicoquímicas.",
        "input": {
            "ph": "Valor del pH del agua.",
            "Hardness": "Dureza del agua.",
            "Solids": "Cantidad total de sólidos disueltos.",
            "Chloramines": "Concentración de cloraminas.",
            "Sulfate": "Concentración de sulfato.",
            "Conductivity": "Conductividad del agua.",
            "Organic_carbon": "Cantidad de carbono orgánico.",
            "Trihalomethanes": "Concentración de trihalometanos.",
            "Turbidity": "Turbidez del agua."
        },
        "output": "1 si el agua es potable, 0 si no lo es."
    }

# Ruta POST para predecir la potabilidad del agua
@app.post("/potabilidad/")
def predict_potability(sample: WaterSample):
    sample_data = np.array([[sample.ph, sample.Hardness, sample.Solids, sample.Chloramines, 
                             sample.Sulfate, sample.Conductivity, sample.Organic_carbon, 
                             sample.Trihalomethanes, sample.Turbidity]])
    
    # Predicción
    prediction = model.predict(sample_data)
    
    return {"potabilidad": int(prediction[0])}

A continuación se muestran algunos requests sobre el servidor generado con uvicorn: <br>
![Ejemplo GET_POST](./imgs/GET_POST.png)

Y la respuesta del servidor con la predicción al POST de ejemplo: <br>
![POST ej](./imgs/POST_ej.png)

# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.

## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

- ¿Cómo se diferencia Docker de una máquina virtual (VM)?
- ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?
- ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?
- ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?
- ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?

A continuación se muestran imágenes que muestran la ejecución del contenedor y los resultados obtenidos: <br>
![Docker build](./imgs/docker_build.png)

Y, por otro lado, la imagen guardada: <br>
![Image saved](./imgs/image_saved.png)

Con respecto a los recursos utilizados por el contenedor, se destaca que este depende del tamaño del modelo y la carga de trabajo, ya que por si mismos no consumen demasiada CPU ni memoria. Ahora bien, la persistencia de datos mediante volúmenes permite que el almacenamiento no se vea afectado, permitiendo así gestionar una gran cantidad de contenedores en un solo host sin demasiada sobrecarga.

A continuación se responden las consultas realizadas por Smapina:
- ¿Cómo se diferencia Docker de una máquina virtual (VM)?
> La principal diferencia entre ambas tecnologías de virtualización es el cómo gestionan los recursos. Por un lado, las máquinas virtuales requieren un sistema operativo completo para operar, consumiendo así más recursos. Por otro lado, Docker utiliza contenedores que comparten el núcleo del sistema operativo del host, es decir, utiliza herramientas más ligeras y rápidas.

- ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?
> Utilizar Docker implica crear un entorno aislado que encapsula todas las dependencias y configuraciones necesarias para que la aplicación funcione, permitiendo así que ésta se ejecute en cualquier entorno de la misma manera, sin problemas de compatibilidad. Por su parte, ejecutar la aplicación directamente en el sistema local puede generar conflictos si el entorno y sus dependencias no coinciden con los de los otros sistemas.

- ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?
> Esto lo realiza al empaquetar la aplicación junto a sus dependencias y configuraciones dentro de un contenedor. De esta forma, la aplicación se ejecutará de la misma forma en cualquier entorno que ejecute dicho contenedor, eliminando posibles diferencias que existan con otros servicios de desarrollo y producción.

- ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?
> Los volúmenes permiten almacenar datos fuera del contenedor, para que así la información no se pierda si es que el contenedor se detiene o elimina. En general, aseguran la integridad de los datos, permitiendo además que sean compartidos entre contenedores o entre contenedor y host.

- ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?
> Dockerfile es un archivo de texto que contiene instrucciones que permiten construir una imagen Docker. Establece las configuraciones del contenedor, las dependencias a instalar y cómo ejecutar las aplicación. Por su parte, el archivo "docker-compose.yml" permite definir y configurar varios contenedores Docker que se espera que se ejecuten juntos, simplificando así la gestión conjunta de estos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<div align="center">
  <img src="https://i.pinimg.com/originals/84/5d/f1/845df1aefc6a5e37ae575327a0cc6e43.gif" width="500">
</div>